In [3]:
!pip install pyreadstat
!pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.4/666.4 kB 7.8 MB/s eta 0:00:00


In [4]:
import pyreadstat
import pandas as pd
import os

In [9]:
# attempted to use pyreadstat to read xpt file
# the  file has invalid type of formatting
df, meta = pyreadstat.read_xport("/content/raw/DEMO_L.xpt")
print(df.head())

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 13: invalid start byte

In [ ]:
#  checking for xpt file that are non readable with pyreadtat because of formatting issue
folder = "/content/raw/"
for file in os.listdir(folder):
    if file.endswith(".xpt"):
        path = os.path.join(folder, file)
        try:
            df, meta = pyreadstat.read_xport(path)
            print(f"✅ Loaded: {file}")
        except Exception as e:
            print(f"❌ Failed: {file} — {e}")

✅ Loaded: VID_L.xpt
✅ Loaded: HEPB_S_L.xpt
✅ Loaded: PAQ_L.xpt
❌ Failed: DEMO_L.xpt — 'utf-8' codec can't decode byte 0x92 in position 13: invalid start byte
✅ Loaded: KIQ_U_L.xpt
✅ Loaded: BPXO_L.xpt
✅ Loaded: WHQ_L.xpt


In [ ]:
# gave up on converting to change formatting issue in the xpt file and tried to use R instead

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 13: invalid start byte

In [1]:
# Using R to convert xpt to csv
# Change Runtime Type to R, prior to doing the script below
# Installing haven (only needed once)
install.packages("haven")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
# Load the library
library(haven)

# Set folder path
folder <- "/content/raw"

In [3]:
# Converting single .xpt file to .csv
library(haven)
data <- read_xpt("/content/raw/DEMO_L.xpt")
write.csv(data, "/content/raw/DEMO_L.csv", row.names = FALSE)

In [ ]:

# Converting all .xpt files to .csv

# List all .xpt files
xpt_files <- list.files(folder, pattern = "\\.xpt$", full.names = TRUE)

# Loop through and convert each to .csv
for (xpt_file in xpt_files) {
  # Read .xpt file
  data <- read_xpt(xpt_file)

  # Create .csv filename
  csv_file <- sub("\\.xpt$", ".csv", xpt_file)

  # Write to CSV
  write.csv(data, csv_file, row.names = FALSE)

  cat("✅ Converted:", xpt_file, "→", csv_file, "\n")
}

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



✅ Converted: /content/raw/BPXO_L.xpt → /content/raw/BPXO_L.csv 
✅ Converted: /content/raw/DEMO_L.xpt → /content/raw/DEMO_L.csv 
✅ Converted: /content/raw/HEPB_S_L.xpt → /content/raw/HEPB_S_L.csv 
✅ Converted: /content/raw/KIQ_U_L.xpt → /content/raw/KIQ_U_L.csv 
✅ Converted: /content/raw/PAQ_L.xpt → /content/raw/PAQ_L.csv 
✅ Converted: /content/raw/VID_L.xpt → /content/raw/VID_L.csv 
✅ Converted: /content/raw/WHQ_L.xpt → /content/raw/WHQ_L.csv 


In [ ]:
# Change Runtime Type back to Python prior to scripts below
# Mapping urls
demo = "/content/raw/DEMO_L.csv"
bp = "/content/raw/BPXO_L.csv"
vitd = "/content/raw/VID_L.csv"
hepb = "/content/raw/HEPB_S_L.csv"
kidney = "/content/raw/KIQ_U_L.csv"
minbehavior = "/content/raw/PAQ_L.csv"
weight = "/content/raw/WHQ_L.csv"

In [2]:
demo = pd.read_csv(demo)
print(demo.head())

NameError: name 'pd' is not defined

In [ ]:
#removing duplicates
demo1 = demo1.drop_duplicates()
